### 🛠️ 1. Initialize notebook variables

❗️ **Run cells 1 & 2 MANUALLY (not via _Run All_)!**

Configures everything that's needed for deployment. 

👉 **Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [ ]:
import utils
import time
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location   = 'eastus2'
index         = 1
deployment    = INFRASTRUCTURE.SIMPLE_APIM
tags          = ['oauth-3rd-party', 'jwt', 'credential-manager', 'policy-fragment']       # ENTER DESCRIPTIVE TAG(S)
api_prefix    = 'oauth-3rd-party-'              # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES
# OAuth
client_id     = 'your-spotify-client-id'        # ENTER THE OAUTH CLIENT ID FOR THE BACKEND API
client_secret = 'your-spotify-client-secret'    # ENTER THE OAUTH CLIENT SECRET FOR THE BACKEND API

# 2) Service-defined parameters (please do not change these)
rg_name       = utils.get_infra_rg_name(deployment, index)
sample_folder = "oauth-3rd-party"
nb_helper     = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM], True)

# 3) Set up the named values
nvs: List[NamedValue] = [
    NamedValue(nb_helper.jwt_key_name, nb_helper.jwt_key_value_bytes_b64, True),
    NamedValue('MarketingMemberRoleId', Role.MARKETING_MEMBER)
]

# 4) Define the APIs and their operations and policies

# Policies
pol_artist_get_xml  = utils.read_policy_xml('artist_get.xml', sample_name = sample_folder)

# Read the policy XML without modifications - it already uses correct APIM named value format
pol_spotify_api_xml = utils.read_and_modify_policy_xml('spotify_api.xml', {
    'jwt_signing_key': '{{' + nb_helper.jwt_key_name + '}}', 
    'marketing_member_role_id': '{{MarketingMemberRoleId}}'
}, sample_folder)  

# Define template parameters for the artists
blob_template_parameters = [
    {
        "name": "id",
        "description": "The Spotify ID of the artist",
        "type": "string",
        "required": True
    }
]

# Spotify
spotify_artist_get = GET_APIOperation2('artists-get', 'Artists', '/artists/{id}', 'Gets the artist by their ID', pol_artist_get_xml, templateParameters = blob_template_parameters)

# APIs Array
apis: List[API] = [
    API(f'{api_prefix}spotify', 'Spotify', f'/{api_prefix}spotify', 'This is the API for interactions with the Spotify REST API', policyXml = pol_spotify_api_xml, operations = [spotify_artist_get], tags = tags),
]

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]},
    'clientId': {'value': client_id},
    'clientSecret': {'value': client_secret}
}

# 2) Deploy the bicep template
output = nb_helper.deploy_bicep(bicep_parameters)

if output.json_data:
    apim_name = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

    # TODO: This should be retrieved from an output; however, the format is static.
    apim_oauth_redirect_url = f'https://authorization-manager.consent.azure-apim.net/redirect/apim/{apim_name}'
    utils.print_val('APIM OAuth Redirect URL', apim_oauth_redirect_url)

utils.print_ok('Deployment completed')

### 🗒️ 3. Authenticate API Management with Spotify

❗️ **The following steps are all manual and cannot presently be automated.**

We have previously created the _APIM_ application in Spotify and have also set Spotify up in Credential Manager via the just-completed bicep. 

#### 3.1 Set Redirect URL in Spotify

Now that the API Management instance has been created, we need to update the redirect URI for the _APIM_ application in Spotify.

1. Open the [Spotify Developer Dashboard](https://developer.spotify.com/dashboard), then click on the _APIM_ application.
1. Press _Edit_ and remove the temporary _localhost_ Redirect URI.
1. Add the `APIM OAuth Redirect URL` (see output above), then press 'Save`. 

#### 3.2 Log API Management into Spotify

We now need to log the _APIM_ application into Spotify via OAuth 2.0.

1. Open the [Azure Portal](https://portal.azure.com) and navigate to your API Management instance.
1. Expand the _APIs_ blade and click on _Credential manager_. You should see the `spotify` credential provider name. Click on it.
1. Press _Connections_. You should see `spotify-auth` with an `Error` status ("This connection is not authenticated.").
1. Click on the ellipsis (...) on the right and select _Login_. This should open a dialog with Spotify, asking you to agree for Spotify and APIM to connect. Press _Agree_.
1. Back in the Azure Portal, press _Refresh_ to see the `Connected` status.


### ✅ 4. Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
import utils
import json
from apimrequests import ApimRequests
from apimtesting import ApimTesting
from users import UserHelper
from authfactory import AuthFactory

tests = ApimTesting("OAuth 3rd Party (Spotify) Sample Tests", sample_folder, deployment)

# 1) Marketing Member Role
# Create a JSON Web Token with a payload and sign it with the symmetric key from above.
encoded_jwt_token_marketing_member = AuthFactory.create_symmetric_jwt_token_for_user(UserHelper.get_user_by_role(Role.MARKETING_MEMBER), nb_helper.jwt_key_value)
print(f'\nJWT token for Marketing Member:\n{encoded_jwt_token_marketing_member}')  # this value is used to call the APIs via APIM

# Issue requests against Front Door.
utils.print_message('Checking if the infrastructure architecture deployment uses Azure Front Door.', blank_above = True)
afd_endpoint_url = utils.get_frontdoor_url(deployment, rg_name)

if afd_endpoint_url:
    artist_id = '06HL4z0CvFAxyc27GXpf02'    # Taylor Swift's Spotify Artist ID
    reqsAfd = ApimRequests(afd_endpoint_url)
    reqsAfd.headers['Authorization'] = f'Bearer {encoded_jwt_token_marketing_member}'
    output = reqsAfd.singleGet(f'/oauth-3rd-party-spotify/artists/{artist_id}', msg = 'Calling the Spotify Artist API via API Management Gateway URL.')
    artist = json.loads(output)
    tests.verify(artist['name'], 'Taylor Swift')
    utils.print_info(f'{artist["name"]} has a popularity rating of {artist["popularity"]} with {artist["followers"]["total"]:,} followers on Spotify.')
else:
    # Issue a direct request to API Management
    artist_id = '6XpaIBNiVzIetEPCWDvAFP'    # Whitney Houston's Spotify Artist ID
    reqsApim = ApimRequests(apim_gateway_url)
    reqsApim.headers['Authorization'] = f'Bearer {encoded_jwt_token_marketing_member}'
    output = reqsApim.singleGet(f'/oauth-3rd-party-spotify/artists/{artist_id}', msg = 'Calling the Spotify Artist API via API Management Gateway URL.')
    artist = json.loads(output)
    tests.verify(artist['name'], 'Whitney Houston')
    utils.print_info(f'{artist["name"]} has a popularity rating of {artist["popularity"]} with {artist["followers"]["total"]:,} followers on Spotify.')

tests.print_summary()

utils.print_ok('All done!')